In [10]:
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
from gaGen import GenGeneticAlgorithm
import copy

In [11]:
def evaluate(X_train : pd.DataFrame, y_train: pd.DataFrame, X_test: pd.DataFrame, y_test: pd.DataFrame, random_state = None):
    
    clf = DecisionTreeClassifier(random_state= random_state)
    
    clf.fit(X_train, y_train)

    predictions = clf.predict(X_test)
    precision = precision_score(predictions,y_test, average='macro')
    f1 = f1_score(predictions,y_test, average='macro')
    recall = recall_score(predictions,y_test, average='macro')
    
    return {"precision":precision, "f1":f1, "recall":recall}

In [12]:
dataset_name= "Liver disorders"

data = pd.read_csv(f"../Datasets/{dataset_name}.csv")

data.replace('?', np.nan, inplace=True)

data.head(10)

,mcv,alkphos,sgpt,sgot,gammagt,drinks,selector
0,85,92,45,27,31,0.0,1
1,85,64,59,32,23,0.0,2
2,86,54,33,16,54,0.0,2
3,91,78,34,24,36,0.0,2
4,87,70,12,28,10,0.0,2
5,98,55,13,17,17,0.0,2
6,88,62,20,17,9,0.5,1
7,88,67,21,11,11,0.5,1
8,92,54,22,20,7,0.5,1
9,90,60,25,19,5,0.5,1


In [13]:
X = data.iloc[:, :-1]

y = data.iloc[:, -1]

In [14]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state= 42)

before_scores= []
selected_scores= []
all_scores = []
columns_number= []

for index,(train_index, test_index) in enumerate(skf.split(X, y)):
    
    fold_x_train =X.iloc[train_index]
    fold_y_train = y.iloc[train_index]
    
    fold_x_test =X.iloc[test_index]
    fold_y_test = y.iloc[test_index]
    
    score_before = evaluate(fold_x_train, fold_y_train, fold_x_test, fold_y_test)
    before_scores.append(score_before)
    
    print(f'''For the original data:
    Columns : {fold_x_train.columns}
    Columns Number : {len(fold_x_train.columns)} 
    precision : {score_before["precision"]}
    f1        : {score_before["f1"]}
    recall    : {score_before["recall"]}
          ''')
    print("Ga Gen started with inputs ......")
    ga= GenGeneticAlgorithm(
        X_train= fold_x_train,
        X_test= fold_x_test,
        y_train= fold_y_train,
        y_test= fold_y_test,
        rnd_state=None
    )

    selected_new_X_train , selected_new_X_test, all__new_X_train ,all_new_X_test = ga.run(population_size= 50, generation_limit= 50, mutation_probability= 0.1)
    
    print("Ga Gen finished ...")

    score_after= evaluate(all__new_X_train, fold_y_train, all_new_X_test, fold_y_test)
    all_scores.append(score_after)
    
    print(f'''\nFor the new generated data with gaGen without selection:
    Columns : {all_new_X_test.columns}
    Columns Number : {len(all_new_X_test.columns)} 
    precision : {score_after["precision"]}
    f1        : {score_after["f1"]}
    recall    : {score_after["recall"]}
    ''')
    score_after= evaluate(selected_new_X_train, fold_y_train, selected_new_X_test, fold_y_test)
    selected_scores.append(score_after)
    
    columns_number.append(len(selected_new_X_test.columns))
    
    print(f'''\nFor the new generated data with gaGen with selection:
    Columns : {selected_new_X_test.columns}
    Columns Number : {len(selected_new_X_test.columns)} 
    precision : {score_after["precision"]}
    f1        : {score_after["f1"]}
    recall    : {score_after["recall"]}
--------------------------------------------------------------------------------------''')
    

For the original data:
    Columns : Index(['mcv', 'alkphos', 'sgpt', 'sgot', 'gammagt', 'drinks'], dtype='object')
    Columns Number : 6 
    precision : 0.6583333333333333
    f1        : 0.6546052631578947
    recall    : 0.6552287581699346
          
Ga Gen started with inputs ......
Ga Gen finished ...

For the new generated data with gaGen without selection:
    Columns : Index(['mcv', 'alkphos', 'sgpt', 'sgot', 'gammagt', 'drinks',
       'mcv_alkphos___addition', 'mcv_alkphos___average',
       'mcv_alkphos___multiplication', 'mcv_sgpt___addition',
       'mcv_sgpt___average', 'mcv_sgpt___multiplication',
       'mcv_sgot___addition', 'mcv_sgot___average',
       'mcv_sgot___multiplication', 'mcv_gammagt___addition',
       'mcv_gammagt___average', 'mcv_gammagt___multiplication',
       'mcv_drinks___addition', 'mcv_drinks___average',
       'mcv_drinks___multiplication', 'alkphos_sgpt___addition',
       'alkphos_sgpt___average', 'alkphos_sgpt___multiplication',
       'alkph

In [15]:
sums = {}
counts = {}

for score in before_scores:
    for key, value in score.items():
        sums[key] = sums.get(key, 0) + value
        counts[key] = counts.get(key, 0) + 1

means = {key: sums[key] / counts[key] for key in sums}

print("Original Data:")
for prop, mean in means.items():
    print(prop + ":", mean)

Original Data:
precision: 0.6658333333333334
f1: 0.6655962640750553
recall: 0.6689621448812626


In [16]:
sums = {}
counts = {}

for score in all_scores:
    for key, value in score.items():
        sums[key] = sums.get(key, 0) + value
        counts[key] = counts.get(key, 0) + 1

means = {key: sums[key] / counts[key] for key in sums}

print("Generated without fs Data:")
for prop, mean in means.items():
    print(prop + ":", mean)

Generated without fs Data:
precision: 0.6160714285714286
f1: 0.613663700404554
recall: 0.623122393129746


In [17]:
sums = {}
counts = {}

for score in selected_scores:
    for key, value in score.items():
        sums[key] = sums.get(key, 0) + value
        counts[key] = counts.get(key, 0) + 1

means = {key: sums[key] / counts[key] for key in sums}

print("Generated with fs Data:")
for prop, mean in means.items():
    print(prop + ":", mean)

Generated with fs Data:
precision: 0.614047619047619
f1: 0.6117949717020321
recall: 0.6153719725836598


In [18]:
print(f"Columns generation: {np.mean(columns_number)}")

Columns generation: 24.3
